In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import shutil
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.visualize as visualize

notebook_name = "cpol_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [157]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2005-11-13T14:00:00"
end = "2005-11-13T16:00:00"

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
cpol_options = data.aura.cpol_data_options(
    start=start, end=end, converted_options=converted_options
)
# Restrict the ERA5 data to a smaller region containing the CPOL radar
lon_range = [129, 133]
lat_range = [-14, -10]
era5_pl_options = data.era5.data_options(
    start=start, end=end, latitude_range=lat_range, longitude_range=lon_range
)
era5_sl_options = data.era5.data_options(
    start=start,
    end=end,
    data_format="single-levels",
    latitude_range=lat_range,
    longitude_range=lon_range,
)
data_options = option.consolidate_options(
    [cpol_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, filename="cpol_era5")

grid_options = grid.create_options(name="geographic")
grid.check_options(grid_options)
grid.save_grid_options(grid_options, filename="cpol_geographic")

# Create the track_options dictionary
track_options = option.mcs(dataset="cpol")
option.save_track_options(track_options, filename="cpol_mcs")

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["mcs"]
}
# visualize_options["middle_cloud"] = visualize.option.runtime_options(
#     "middle_cloud", save=True, style="presentation", figure_types=["mask"]
# )
# visualize.option.save_display_options(visualize_options, filename="runtime_mcs")
visualize_options = None

2024-09-03 18:33:18,359 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/data_options/cpol_era5.yaml
2024-09-03 18:33:18,429 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-09-03 18:33:18,430 - thor.grid - WARNING - Coordinates not specified. Will attempt to infer from input.
2024-09-03 18:33:18,434 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/grid_options/cpol_geographic.yaml
2024-09-03 18:33:18,469 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/track_options/cpol_mcs.yaml


In [158]:
output_directory = base_local / "runs/cpol_demo_geographic"
if output_directory.exists():
    shutil.rmtree(output_directory)
times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory
)

2024-09-03 18:33:19,675 - thor.track - INFO - Beginning simultaneous tracking.
2024-09-03 18:33:19,785 - thor.track - INFO - Processing 2005-11-13T14:00:09.
2024-09-03 18:33:19,788 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T14:00:09.
2024-09-03 18:33:19,790 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.140000.nc
2024-09-03 18:33:19,883 - thor.data.aura - INFO - Creating new geographic grid with spacing 0.025 m, 0.025 m.
2024-09-03 18:33:20,642 - thor.track - INFO - Processing hierarchy level 0.
2024-09-03 18:33:20,644 - thor.track - INFO - Tracking cell.
2024-09-03 18:33:20,663 - thor.match.match - INFO - Matching cell objects.
2024-09-03 18:33:20,666 - thor.match.match - INFO - No previous mask, or no objects in previous mask.
2024-09-03 18:33:20,672 - thor.visualize.runtime - INFO - Generating runtime visualizations.
2024-09-03 18:33:20,674 - thor.attribute.attribute - INFO - Recording object attributes.
2024-09-03 18:33:20,675 -

KeyError: <class 'float'>

In [151]:
import pandas as pd
import thor.attribute.utils as utils
df = utils.read_attribute_csv(output_directory / f"attributes/cell/core.csv")

In [156]:
df

,level_0,index,time,universal_id,latitude,longitude,area,u_flow,v_flow,u_displacement,v_displacement
0,0,0,2005-11-13 14:10:23,1,-12.375,131.675003,75.199997,-6.6,2.3,-4.400000,-0.0
1,1,1,2005-11-13 14:10:23,2,-12.150,131.774994,481.600006,-6.6,2.3,-4.400000,-0.0
2,2,2,2005-11-13 14:20:09,1,-12.375,131.649994,90.199997,-4.6,-0.0,NaN,NaN
3,3,3,2005-11-13 14:20:09,2,-12.150,131.750000,466.600006,-4.6,-0.0,-9.300000,-0.0
4,4,4,2005-11-13 14:20:09,3,-12.300,131.550003,52.700001,-2.3,2.4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
85,85,85,2005-11-13 15:50:09,13,-12.200,131.050003,30.100000,-9.1,-0.0,-4.500000,-0.0
86,86,86,2005-11-13 16:00:08,2,-12.125,131.125000,549.400024,-9.1,4.6,-9.100000,9.2
87,87,87,2005-11-13 16:00:08,8,-11.750,131.425003,105.500000,-4.5,-0.0,-4.500000,4.6
88,88,88,2005-11-13 16:00:08,9,-12.450,131.375000,293.200012,-4.5,-0.0,-9.100000,-0.0


# Cartesian Coordinates

In [63]:
grid_options = grid.create_options(name="cartesian", regrid=False)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, filename="cpol_cartesian")

2024-09-03 14:32:03,096 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-09-03 14:32:03,100 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/grid_options/cpol_cartesian.yaml


In [66]:
output_directory = base_local / "runs/cpol_demo_cartesian"
if output_directory.exists():
    shutil.rmtree(output_directory)
times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory,
)

2024-09-03 15:04:19,440 - thor.track - INFO - Beginning simultaneous tracking.
2024-09-03 15:04:19,600 - thor.track - INFO - Processing 2005-11-13T14:00:09.
2024-09-03 15:04:19,603 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T14:00:09.
2024-09-03 15:04:19,604 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.140000.nc
2024-09-03 15:04:19,830 - thor.track - INFO - Processing hierarchy level 0.
2024-09-03 15:04:19,831 - thor.track - INFO - Tracking cell.
2024-09-03 15:04:19,853 - thor.match.match - INFO - Matching cell objects.
2024-09-03 15:04:19,855 - thor.match.match - INFO - No previous mask, or no objects in previous mask.
2024-09-03 15:04:19,862 - thor.visualize.runtime - INFO - Generating runtime visualizations.
2024-09-03 15:04:19,863 - thor.attribute.attribute - INFO - Recording object attributes.
2024-09-03 15:04:19,865 - thor.track - INFO - Tracking middle_cloud.
2024-09-03 15:04:19,875 - thor.visualize.runtime - INFO - Generati